In [1]:
import pickle
import pandas as pd

from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from typing import Callable
from src.config import Config
from src.dataset import DatasetProcessor
from src.metrics import PrecisionAtK, RecallAtK
from src.optimizer import OptunaMaximizer

In [2]:
with Config.PREPROCESSED_INTERACTIONS_PATH.open("rb") as file_object:
    interactions = pickle.load(file_object)

In [3]:
interactions

,user_id,item_id,progress,rating,start_date,interest_score
0,126706,14433,80,NaN,2018-01-01,0.400
1,127290,140952,58,NaN,2018-01-01,0.290
2,66991,198453,89,NaN,2018-01-01,0.445
3,46791,83486,23,5.0,2018-01-01,0.615
4,79313,188770,88,5.0,2018-01-01,0.940
...,...,...,...,...,...,...
1532993,153908,98585,44,NaN,2019-02-11,0.220
1532994,154008,251969,4,NaN,2018-04-08,0.020
1532995,154892,298192,68,5.0,2019-02-16,0.840
1532996,156948,38118,78,5.0,2018-08-19,0.890


In [4]:
train, test = DatasetProcessor.split_train_test(interactions, "start_date", Config.TEST_DAYS)
train, valid = DatasetProcessor.split_train_test(train, "start_date", Config.TEST_DAYS * 2)
train = train[["user_id", "item_id", "interest_score"]]
test = test[["user_id", "item_id", "interest_score"]]
valid = valid[["user_id", "item_id", "interest_score"]]

In [5]:
train

,user_id,item_id,interest_score
0,126706,14433,0.400
1,127290,140952,0.290
2,66991,198453,0.445
3,46791,83486,0.615
4,79313,188770,0.940
...,...,...,...
1532993,153908,98585,0.220
1532994,154008,251969,0.020
1532995,154892,298192,0.840
1532996,156948,38118,0.890


In [6]:
test

,user_id,item_id,interest_score
1517914,38753,135245,0.000
1517915,101642,319500,0.835
1517916,13548,251184,0.000
1517917,130425,193445,0.490
1517918,93986,80733,0.235
...,...,...,...
1530838,141930,219928,0.450
1530839,53358,42887,0.290
1530840,151170,284652,0.135
1530841,141293,273421,0.240


In [7]:
valid

,user_id,item_id,interest_score
1503047,22032,287219,0.275
1503048,84214,121609,1.000
1503049,28992,11482,0.110
1503050,23345,281921,1.000
1503051,49466,2880,0.010
...,...,...,...
1517909,138587,291806,0.000
1517910,158991,99669,0.815
1517911,77232,142149,0.020
1517912,17843,174535,0.060


In [8]:
csr_train = csr_matrix(
    (train["interest_score"], (train["user_id"], train["item_id"]))
)
csr_train

<159613x321752 sparse matrix of type '<class 'numpy.float32'>'
	with 1505202 stored elements in Compressed Sparse Row format>

In [9]:
class Objective:
    def __init__(self, train: csr_matrix, valid: pd.DataFrame, metric: Callable):
        self.train = train
        self.valid = valid
        self.metric = metric

    def __call__(self, trial) -> float:
        search_space = {
            "factors": trial.suggest_int("factors", 4, 256),
            "regularization": trial.suggest_float("regularization", 1e-8, 0.1),
            "iterations": trial.suggest_int("iterations", 8, 64)
        }
        model = AlternatingLeastSquares(**search_space)
        model.fit(self.train)
        predictions = pd.DataFrame(self.valid["user_id"].drop_duplicates())
        predictions["item_id"] = predictions["user_id"].apply(
            lambda user_id: 
                model.recommend(
                    user_id, 
                    self.train[user_id],
                    N=Config.K,
                    filter_already_liked_items=True
                )[0]
        )
        predictions = predictions.explode("item_id")
        return self.valid["user_id"].apply(
            lambda user: 
                self.metric.calculate(
                    predictions[predictions["user_id"] == user]["item_id"],
                    test[test["user_id"] == user]["item_id"]
                )
        ).mean()

In [10]:
recall = RecallAtK(Config.K)
precision = PrecisionAtK(Config.K)

In [11]:
precision_objective = Objective(csr_train, valid, precision)
precision_optimizer = OptunaMaximizer(precision_objective, "ALS precision maximize")
precision_best_trial = precision_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-21 21:14:18,412] A new study created in memory with name: ALS precision maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

[I 2023-02-21 21:14:56,715] Trial 0 finished with value: 0.0024752808232999333 and parameters: {'factors': 59, 'regularization': 0.03583772580478882, 'iterations': 60}. Best is trial 0 with value: 0.0024752808232999333.


  0%|          | 0/16 [00:00<?, ?it/s]

...

[I 2023-02-21 21:25:21,115] Trial 10 finished with value: 0.0028452276854779193 and parameters: {'factors': 183, 'regularization': 0.05447786856628104, 'iterations': 8}. Best is trial 2 with value: 0.002999932736934167.


  0%|          | 0/21 [00:00<?, ?it/s]

[I 2023-02-21 21:25:37,995] Trial 11 finished with value: 0.002912490751328462 and parameters: {'factors': 178, 'regularization': 0.0488676091827696, 'iterations': 21}. Best is trial 2 with value: 0.002999932736934167.


  0%|          | 0/44 [00:00<?, ?it/s]

[I 2023-02-21 21:25:58,472] Trial 12 finished with value: 0.002650164794511349 and parameters: {'factors': 187, 'regularization': 0.05295882610394445, 'iterations': 44}. Best is trial 2 with value: 0.002999932736934167.
EarlyStopping Exceeded: No new best scores on iters 10


In [12]:
recall_objective = Objective(csr_train, valid, recall)
recall_optimizer = OptunaMaximizer(recall_objective, "ALS recall maximize")
recall_best_trial = recall_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-02-21 21:25:58,563] A new study created in memory with name: ALS recall maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

[I 2023-02-21 21:26:10,250] Trial 0 finished with value: 0.007753072489322144 and parameters: {'factors': 65, 'regularization': 0.0948623418988534, 'iterations': 19}. Best is trial 0 with value: 0.007753072489322144.


  0%|          | 0/17 [00:00<?, ?it/s]

...

[I 2023-02-21 21:34:46,233] Trial 29 finished with value: 0.008596569234223101 and parameters: {'factors': 190, 'regularization': 0.09985623744814402, 'iterations': 53}. Best is trial 21 with value: 0.011404314503460246.


  0%|          | 0/8 [00:00<?, ?it/s]

[I 2023-02-21 21:35:01,903] Trial 30 finished with value: 0.009020726924473475 and parameters: {'factors': 159, 'regularization': 0.09053485732499203, 'iterations': 8}. Best is trial 21 with value: 0.011404314503460246.


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-02-21 21:35:25,758] Trial 31 finished with value: 0.010030910436170402 and parameters: {'factors': 251, 'regularization': 0.07871978908718259, 'iterations': 25}. Best is trial 21 with value: 0.011404314503460246.
EarlyStopping Exceeded: No new best scores on iters 10
